In [ ]:
!pip install --upgrade google-cloud-vision
!pip install opencv-python matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.9/527.9 kB 26.7 MB/s eta 0:00:00


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/Energy7/Track_detection/internship-energy7-4d187921feea.json"

In [ ]:
from google.cloud import vision
import json
from PIL import Image

input_folder = '/content/drive/MyDrive/Energy7/Week11/Dataset'
ocr_output = {}

client = vision.ImageAnnotatorClient()

for fname in sorted(os.listdir(input_folder)):
    if not fname.lower().endswith((".jpg", ".png", ".jpeg")):
        continue

    image_path = os.path.join(input_folder, fname)
    print(f"Processing OCR for: {fname}")

    with open(image_path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations

    ocr_output[fname] = []
    for t in texts[1:]:
        box = [(v.x, v.y) for v in t.bounding_poly.vertices]
        ocr_output[fname].append({
            "text": t.description,
            "box": box
        })

# Save JSON
with open("/content/drive/MyDrive/Energy7/Week11/track_ocr_results.json", "w") as f:
    json.dump(ocr_output, f, indent=2)

print("✅ OCR completed and saved as track_ocr_results.json")

Processing OCR for: Screenshot 2025-06-19 182939.png
Processing OCR for: Screenshot 2025-06-19 183530.png
Processing OCR for: Screenshot 2025-06-19 184543.png
Processing OCR for: Screenshot 2025-06-19 185239.png
Processing OCR for: Screenshot 2025-06-19 185502.png
Processing OCR for: Screenshot 2025-06-19 185840.png
Processing OCR for: Screenshot 2025-06-19 190410.png
Processing OCR for: Screenshot 2025-06-19 191835.png
Processing OCR for: Screenshot 2025-06-19 192015.png
Processing OCR for: Screenshot 2025-06-19 192146.png
Processing OCR for: Screenshot 2025-06-19 192245.png
Processing OCR for: Screenshot 2025-06-19 192420.png
Processing OCR for: Screenshot 2025-06-19 193207.png
Processing OCR for: Screenshot 2025-06-19 193301.png
Processing OCR for: Screenshot 2025-06-19 193511.png
Processing OCR for: Screenshot 2025-06-19 193725.png
Processing OCR for: Screenshot 2025-06-19 193857.png
Processing OCR for: Screenshot 2025-06-19 194044.png
✅ OCR completed and saved as track_ocr_results

In [ ]:
import json

# Load COCO JSON
with open("/content/drive/MyDrive/Energy7/Week11/_annotations.coco.json") as f:
    coco = json.load(f)

# Build image_id to cleaned filename mapping using 'extra.name'
id_to_file = {
    img["id"]: img["extra"]["name"] for img in coco["images"]
    if "extra" in img and "name" in img["extra"]
}

# Map category IDs to names
categories = {cat["id"]: cat["name"] for cat in coco["categories"]}

# Prepare final output
output = {}

# Parse annotations
for ann in coco["annotations"]:
    img_id = ann["image_id"]
    label = categories.get(ann["category_id"], "unknown")
    x, y, w, h = ann["bbox"]
    x1, y1 = int(x), int(y)
    x2, y2 = int(x + w), int(y + h)

    img_name = id_to_file.get(img_id)
    if not img_name:
        continue  # Skip if no name found

    if img_name not in output:
        output[img_name] = []

    output[img_name].append({
        "label": label,
        "x1": x1,
        "y1": y1,
        "x2": x2,
        "y2": y2
    })

# Save to file
output_path = "/content/drive/MyDrive/Energy7/Week11/roboflow_gt_bboxes.json"
with open(output_path, "w") as f:
    json.dump(output, f, indent=2)

print("✅ Converted Roboflow COCO JSON to usable JSON format")


✅ Converted Roboflow COCO JSON to usable JSON format


In [ ]:
import json
import os
from collections import defaultdict

# ---------- CONFIG ----------

image_filename = "Screenshot 2025-06-19 182939.png"
image_key_in_bboxes = "Screenshot-2025-06-19-182939.png"

bbox_json_path = "/content/drive/MyDrive/Energy7/Week11/roboflow_gt_bboxes.json"
ocr_json_path = "/content/drive/MyDrive/Energy7/Week11/track_ocr_results.json"
save_dir = "/content/drive/MyDrive/Energy7/Week11/Family_Connections_bbox/"
os.makedirs(save_dir, exist_ok=True)

# ---------- Load JSON ----------

with open(bbox_json_path) as f:
    track_boxes_raw = json.load(f)

with open(ocr_json_path) as f:
    ocr_data_raw = json.load(f)

track_boxes = track_boxes_raw[image_key_in_bboxes]
ocr_entries = ocr_data_raw[image_filename]

# ---------- Utility Functions ----------

def get_center(box):
    x_coords = [pt[0] for pt in box]
    y_coords = [pt[1] for pt in box]
    return sum(x_coords) // 4, sum(y_coords) // 4

def box_key(box_tuple):
    return "_".join(str(int(round(v))) for v in box_tuple)

# ---------- Step 1: Match OCR Labels to Track Boxes ----------

ocr_labels = []
for entry in ocr_entries:
    text = entry["text"]
    if text.endswith("T") and len(text) <= 5:
        cx, cy = get_center(entry["box"])
        ocr_labels.append({"text": text, "cx": cx, "cy": cy, "box": entry["box"]})

label_to_box = {}
for label in ocr_labels:
    best_box = None
    min_dist = float('inf')
    for box in track_boxes:
        if not (box["y1"] - 30 <= label["cy"] <= box["y2"] + 30):
            continue
        box_cx = (box["x1"] + box["x2"]) // 2
        dist = abs(label["cx"] - box_cx)
        if dist < min_dist:
            min_dist = dist
            best_box = (box["x1"], box["y1"], box["x2"], box["y2"])
    if best_box:
        key = box_key(best_box)
        label_to_box[label["text"]] = key

# ---------- Step 2: Save as JSON ----------

# ✅ Save OCR-label → bounding box mapping as _ocr_bbox_mapping.json
ocr_box_output = defaultdict(list)

for label, key in label_to_box.items():
    x1, y1, x2, y2 = map(int, key.split("_"))
    ocr_box_output[image_key_in_bboxes].append({
        "label": label,
        "x1": x1,
        "y1": y1,
        "x2": x2,
        "y2": y2
    })

# Save file
ocr_bbox_json_path = os.path.join(save_dir, image_filename.replace(".png", "_ocr_bbox_mapping.json"))
with open(ocr_bbox_json_path, "w") as f:
    json.dump(ocr_box_output, f, indent=2)

print(f"✅ OCR-BoundingBox mapping saved to: {ocr_bbox_json_path}")


✅ OCR-BoundingBox mapping saved to: /content/drive/MyDrive/Energy7/Week11/Family_Connections_bbox/Screenshot 2025-06-19 182939_ocr_bbox_mapping.json


In [ ]:
import json
import cv2
import os
from collections import defaultdict

# ------------------ CONFIG ------------------

image_key = "Screenshot-2025-06-19-182939.png"
image_file = "Screenshot 2025-06-19 182939.png"

mapped_boxes_path = "/content/drive/MyDrive/Energy7/Week11/Family_Connections_bbox/Screenshot 2025-06-19 182939_ocr_bbox_mapping.json"
image_path = f"/content/drive/MyDrive/Energy7/Week11/Dataset/{image_file}"
save_dir = "/content/drive/MyDrive/Energy7/Week11/Family_Connections_bbox/"
os.makedirs(save_dir, exist_ok=True)

# ------------------ PARAMETERS ------------------

HORIZONTAL_GAP_THRESHOLD = 20
VERTICAL_OVERLAP_THRESHOLD = 10

# ------------------ STEP 1: LOAD DATA ------------------

with open(mapped_boxes_path) as f:
    all_data = json.load(f)

track_boxes = all_data[image_key]

# ------------------ STEP 2: BUILD CONNECTIONS ------------------

connections = defaultdict(list)

for i, box1 in enumerate(track_boxes):
    for j, box2 in enumerate(track_boxes):
        if i == j:
            continue

        label1, x1a, y1a, x2a, y2a = box1["label"], box1["x1"], box1["y1"], box1["x2"], box1["y2"]
        label2, x1b, y1b, x2b, y2b = box2["label"], box2["x1"], box2["y1"], box2["x2"], box2["y2"]

        # Check horizontal adjacency
        horizontal_touch = abs(x2a - x1b) <= HORIZONTAL_GAP_THRESHOLD or abs(x1a - x2b) <= HORIZONTAL_GAP_THRESHOLD

        # Check vertical overlap (i.e., they lie in the same row)
        vertical_overlap = min(y2a, y2b) - max(y1a, y1b) >= -VERTICAL_OVERLAP_THRESHOLD

        if horizontal_touch and vertical_overlap:
            if label2 not in connections[label1]:
                connections[label1].append(label2)
            if label1 not in connections[label2]:
                connections[label2].append(label1)

# ------------------ STEP 3: SAVE CONNECTIONS JSON ------------------

json_output_path = os.path.join(save_dir, image_key.replace(".jpg", "_connections.json"))
with open(json_output_path, "w") as f:
    json.dump(connections, f, indent=2)
print(f"✅ Connections saved to: {json_output_path}")

# ------------------ STEP 4: PLOT AND SAVE VISUAL ------------------

img = cv2.imread(image_path)

# Draw bounding boxes
for box in track_boxes:
    x1, y1, x2, y2 = box["x1"], box["y1"], box["x2"], box["y2"]
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cx = (x1 + x2) // 2
    cy = (y1 + y2) // 2
    cv2.putText(img, box["label"], (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    box["center"] = (cx, cy)  # Save for line drawing

# Draw connection lines
label_to_center = {box["label"]: box["center"] for box in track_boxes if "center" in box}

for label1, neighbors in connections.items():
    for label2 in neighbors:
        pt1 = label_to_center.get(label1)
        pt2 = label_to_center.get(label2)
        if pt1 and pt2:
            cv2.line(img, pt1, pt2, (255, 0, 0), 2)

visual_path = os.path.join(save_dir, image_key.replace(".jpg", "_connections.png"))
cv2.imwrite(visual_path, img)
print(f"✅ Visual saved to: {visual_path}")


✅ Connections saved to: /content/drive/MyDrive/Energy7/Week11/Family_Connections_bbox/Screenshot-2025-06-19-182939.png
✅ Visual saved to: /content/drive/MyDrive/Energy7/Week11/Family_Connections_bbox/Screenshot-2025-06-19-182939.png


# Bounding box for all images

In [1]:
import json
import os
from collections import defaultdict

# ---------- CONFIG ----------
bbox_json_path = "/content/drive/MyDrive/Energy7/Week11/Horizontal Final/roboflow_gt_bboxes.json"
ocr_json_path = "/content/drive/MyDrive/Energy7/Week12/track_ocr_results.json"
save_dir = "/content/drive/MyDrive/Energy7/Week11/Horizontal Final/Final_Mapped_bboxes/"
os.makedirs(save_dir, exist_ok=True)

# ---------- Load JSON ----------
with open(bbox_json_path) as f:
    track_boxes_raw = json.load(f)

with open(ocr_json_path) as f:
    ocr_data_raw = json.load(f)

# ---------- Utility Functions ----------
def get_center(box):
    x_coords = [pt[0] for pt in box]
    y_coords = [pt[1] for pt in box]
    return sum(x_coords) // 4, sum(y_coords) // 4

def box_key(box_tuple):
    return "_".join(str(int(round(v))) for v in box_tuple)

# ---------- Process All Images ----------
for image_filename in ocr_data_raw.keys():
    image_key_in_bboxes = image_filename.replace(" ", "-")

    if image_key_in_bboxes not in track_boxes_raw:
        print(f"⚠️ Skipping: No track boxes found for {image_filename}")
        continue

    track_boxes = track_boxes_raw[image_key_in_bboxes]
    ocr_entries = ocr_data_raw[image_filename]

    ocr_labels = []
    for entry in ocr_entries:
        text = entry["text"]
        if text.endswith("T") and len(text) <= 5:
            cx, cy = get_center(entry["box"])
            ocr_labels.append({"text": text, "cx": cx, "cy": cy, "box": entry["box"]})

    label_to_box = {}
    box_to_label = {}  # <--- new

    for label in ocr_labels:
        best_box = None
        min_dist = float('inf')
        for box in track_boxes:
            if not (box["y1"] - 30 <= label["cy"] <= box["y2"] + 30):
                continue
            box_cx = (box["x1"] + box["x2"]) // 2
            dist = abs(label["cx"] - box_cx)
            if dist < min_dist:
                min_dist = dist
                best_box = (box["x1"], box["y1"], box["x2"], box["y2"])
        if best_box:
            key = box_key(best_box)
            label_to_box[label["text"]] = key
            box_to_label[key] = label["text"]

    final_output = defaultdict(list)
    unknown_count = 1

    for box in track_boxes:
        key = box_key((box["x1"], box["y1"], box["x2"], box["y2"]))
        label = box_to_label.get(key, f"unknown_track_{unknown_count}")
        if label.startswith("unknown_track"):
            unknown_count += 1

        final_output[image_key_in_bboxes].append({
            "label": label,
            "x1": box["x1"],
            "y1": box["y1"],
            "x2": box["x2"],
            "y2": box["y2"]
        })

    # Save file
    save_path = os.path.join(save_dir, image_filename.replace(".png", "_ocr_bbox_mapping.json"))
    with open(save_path, "w") as f:
        json.dump(final_output, f, indent=2)

    print(f"✅ Final mapping saved for {image_filename}")


✅ Final mapping saved for Screenshot 2025-06-19 182939.png
⚠️ Skipping: No track boxes found for Screenshot 2025-06-19 183530.png
⚠️ Skipping: No track boxes found for Screenshot 2025-06-19 184543.png
✅ Final mapping saved for Screenshot 2025-06-19 185239.png
⚠️ Skipping: No track boxes found for Screenshot 2025-06-19 185502.png
⚠️ Skipping: No track boxes found for Screenshot 2025-06-19 185840.png
✅ Final mapping saved for Screenshot 2025-06-19 190410.png
⚠️ Skipping: No track boxes found for Screenshot 2025-06-19 191835.png
⚠️ Skipping: No track boxes found for Screenshot 2025-06-19 192015.png
✅ Final mapping saved for Screenshot 2025-06-19 192146.png
⚠️ Skipping: No track boxes found for Screenshot 2025-06-19 192245.png
⚠️ Skipping: No track boxes found for Screenshot 2025-06-19 192420.png
⚠️ Skipping: No track boxes found for Screenshot 2025-06-19 193207.png
⚠️ Skipping: No track boxes found for Screenshot 2025-06-19 193301.png
⚠️ Skipping: No track boxes found for Screenshot 2025-

In [2]:
import json
import cv2
import os
from collections import defaultdict

# ------------------ CONFIG ------------------
ocr_mapping_folder = "/content/drive/MyDrive/Energy7/Week11/Horizontal Final/Final_Mapped_bboxes/"
image_folder = "/content/drive/MyDrive/Energy7/Week11/Dataset/"
save_dir = "/content/drive/MyDrive/Energy7/Week11/Horizontal Final/Output_horizontal_connections/"
os.makedirs(save_dir, exist_ok=True)

# ------------------ PARAMETERS ------------------
HORIZONTAL_GAP_THRESHOLD = 20
VERTICAL_OVERLAP_THRESHOLD = 10

# ------------------ STEP: Loop Through All Mapping Files ------------------
for filename in os.listdir(ocr_mapping_folder):
    if not filename.endswith("_ocr_bbox_mapping.json"):
        continue

    json_path = os.path.join(ocr_mapping_folder, filename)
    image_key = filename.replace("_ocr_bbox_mapping.json", ".png")
    image_path = os.path.join(image_folder, image_key)

    with open(json_path) as f:
        all_data = json.load(f)

    if not all_data:
        print(f"⚠️ Skipping empty file: {filename}")
        continue

    image_key_internal = list(all_data.keys())[0]
    track_boxes = all_data[image_key_internal]

    # ------------------ STEP 2: BUILD CONNECTIONS ------------------
    connections = defaultdict(list)

    for i, box1 in enumerate(track_boxes):
        for j, box2 in enumerate(track_boxes):
            if i == j:
                continue

            label1, x1a, y1a, x2a, y2a = box1["label"], box1["x1"], box1["y1"], box1["x2"], box1["y2"]
            label2, x1b, y1b, x2b, y2b = box2["label"], box2["x1"], box2["y1"], box2["x2"], box2["y2"]

            horizontal_touch = abs(x2a - x1b) <= HORIZONTAL_GAP_THRESHOLD or abs(x1a - x2b) <= HORIZONTAL_GAP_THRESHOLD
            vertical_overlap = min(y2a, y2b) - max(y1a, y1b) >= -VERTICAL_OVERLAP_THRESHOLD

            if horizontal_touch and vertical_overlap:
                if label2 not in connections[label1]:
                    connections[label1].append(label2)
                if label1 not in connections[label2]:
                    connections[label2].append(label1)

    # ------------------ STEP 3: SAVE CONNECTIONS JSON ------------------
    json_output_path = os.path.join(save_dir, image_key.replace(".png", "_connections.json"))
    with open(json_output_path, "w") as f:
        json.dump(connections, f, indent=2)
    print(f"✅ Connections saved to: {json_output_path}")

    # ------------------ STEP 4: PLOT AND SAVE VISUAL ------------------
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Could not read image: {image_path}")
        continue

    # Draw bounding boxes
    for box in track_boxes:
        x1, y1, x2, y2 = box["x1"], box["y1"], box["x2"], box["y2"]
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2
        cv2.putText(img, box["label"], (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        box["center"] = (cx, cy)

    label_to_center = {box["label"]: box["center"] for box in track_boxes if "center" in box}

    for label1, neighbors in connections.items():
        for label2 in neighbors:
            pt1 = label_to_center.get(label1)
            pt2 = label_to_center.get(label2)
            if pt1 and pt2:
                cv2.line(img, pt1, pt2, (255, 0, 0), 2)

    visual_path = os.path.join(save_dir, image_key.replace(".png", "_connections.png"))
    cv2.imwrite(visual_path, img)
    print(f"✅ Visual saved to: {visual_path}")


✅ Connections saved to: /content/drive/MyDrive/Energy7/Week11/Horizontal Final/Output_horizontal_connections/Screenshot 2025-06-19 182939_connections.json
✅ Visual saved to: /content/drive/MyDrive/Energy7/Week11/Horizontal Final/Output_horizontal_connections/Screenshot 2025-06-19 182939_connections.png
✅ Connections saved to: /content/drive/MyDrive/Energy7/Week11/Horizontal Final/Output_horizontal_connections/Screenshot 2025-06-19 185239_connections.json
✅ Visual saved to: /content/drive/MyDrive/Energy7/Week11/Horizontal Final/Output_horizontal_connections/Screenshot 2025-06-19 185239_connections.png
✅ Connections saved to: /content/drive/MyDrive/Energy7/Week11/Horizontal Final/Output_horizontal_connections/Screenshot 2025-06-19 190410_connections.json
✅ Visual saved to: /content/drive/MyDrive/Energy7/Week11/Horizontal Final/Output_horizontal_connections/Screenshot 2025-06-19 190410_connections.png
✅ Connections saved to: /content/drive/MyDrive/Energy7/Week11/Horizontal Final/Output_ho